<a href="https://colab.research.google.com/github/aminojagh/fast-ai/blob/main/NB2_How_does_a_neural_net_really_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fitting a function with gradient descent

In [14]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import torch, numpy as np
from functools import partial

# from fastai.basics import *

In [15]:
plt.rc('figure', dpi=90)

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim);
    plt.plot(x, f(x), color);
    if title is not None: plt.title(title);

In [16]:
def quad(a, b, c, x): return a*x**2 + b*x + c
def mk_quad(a,b,c): return partial(quad, a,b,c)
def noise(x, scale): return np.random.normal(scale=scale, size=x.shape)
def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)
def mae(preds, acts): return (torch.abs(preds-acts)).mean()

def _plot_quad(x, y, a, b, c):
    f = mk_quad(a,b,c)
    plt.scatter(x,y)
    loss = mae(f(x), y)
    plot_function(f, ylim=(-3,12), title=f"MAE: {loss:.2f}")

In [21]:
np.random.seed(42)
f2 = mk_quad(3,2,1)
x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f2(x), 0.15, 1.5)

@interact(a=1.1, b=1.1, c=1.1)
def plot_quad(a,b,c): return partial(_plot_quad, x,y)(a,b,c)

<ipython-input-16-d77a0d107243>:4: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)


interactive(children=(FloatSlider(value=1.1, description='a', max=3.3000000000000003, min=-1.1), FloatSlider(v…

## Automating gradient descent

In [22]:
def quad_mae(params):
    f = mk_quad(*params)
    return mae(f(x), y)

In [23]:
abc = torch.tensor([1.1,1.1,1.1])
abc.requires_grad_()
loss = quad_mae(abc)
print(f"loss: {loss}")
loss.backward()
print(f"grdients of parameters: {abc.grad}")

loss: 2.421854506915183
grdients of parameters: tensor([-1.3529, -0.0316, -0.5000])


In [24]:
for i in range(10):
    loss = quad_mae(abc)
    loss.backward()
    with torch.no_grad(): abc -= abc.grad*0.01
    print(f'step={i}; loss={loss:.2f}')

step=0; loss=2.42
step=1; loss=2.38
step=2; loss=2.32
step=3; loss=2.23
step=4; loss=2.13
step=5; loss=2.01
step=6; loss=1.87
step=7; loss=1.73
step=8; loss=1.60
step=9; loss=1.47


## How a neural network approximates any given function

In [28]:
import torch.nn.functional as F

def rectified_linear(m,b,x): return torch.clip(m*x+b, 0.)
def rectified_linear2(m,b,x): return F.relu(m*x+b)

def double_relu(m1,b1,m2,b2,x):
    return rectified_linear(m1,b1,x) + rectified_linear(m2,b2,x)

In [31]:
# @interact(m=1.5, b=1.5)
# def plot_relu(m, b):
#     plot_function(partial(rectified_linear, m,b), ylim=(-1,4))

@interact(m1=-1.5, b1=-1.5, m2=1.5, b2=1.5)
def plot_double_relu(m1, b1, m2, b2):
    plot_function(partial(double_relu, m1,b1,m2,b2), ylim=(-1,6))

interactive(children=(FloatSlider(value=-1.5, description='m1', max=1.5, min=-4.5), FloatSlider(value=-1.5, de…